In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain_community.document_loaders.dataframe import DataFrameLoader
# from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from IPython.display import display, Markdown
from langchain_openai import AzureOpenAIEmbeddings

In [3]:
llm = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0.9
                        )

/tmp/ipykernel_22182/1810801969.py:1: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(api_key=azure_openai_api_key,


In [5]:
embeddings = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
                                    azure_deployment='text-embedding-3-large',
                                    azure_endpoint=azure_openai_api_endpoint,
                                    openai_api_version="2023-05-15",
                                    chunk_size=500
)


# vectorstore = FAISS.from_documents(documents, embeddings)
# vectorstore

In [21]:
import faiss
index = faiss.read_index("correction/faiss_vector_store/index.faiss", faiss.IO_FLAG_MMAP | faiss.IO_FLAG_READ_ONLY)
index.search(,k=2)

AttributeError: 'int' object has no attribute 'shape'

In [8]:
vectorstore = FAISS.load_local('correction/faiss_vector_store', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.1})
docs = retriever.invoke('When did the author of "L\'assommoir" die?')
docs

[Document(metadata={'source': 'gutenberg_ebooks_modified.csv', 'row': 0}, page_content=': 0\nindex: 0\nEbook ID: 8600\nAuthor: Zola, Émile, 1840-1902\nTitle: L\'Assommoir\nCredits: John Bickers, Dagny and David Widger\nSummary: "L\'Assommoir" by Émile Zola is a novel written during the late 19th century, an era characterized by the realism movement in literature. The book explores the struggles of Gervaise, a laundress trying to build a life for herself and her children amidst the oppressive and often brutal conditions of working-class Paris. The story highlights themes of poverty, domestic strife, and the impact of alcoholism on individuals and families.  The opening of "L\'Assommoir" presents Gervaise in a state of despair, anxiously awaiting the return of Lantier, her partner, who has been increasingly unfaithful and irresponsible. The narrative depicts her emotional turmoil as she reflects on their meager living conditions and the struggles of single motherhood. Gervaise\'s observa

In [14]:
result = await vectorstore.asimilarity_search(
    'When did the author of "L\'assommoir" die?',
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1},
    k=2)

result

[Document(metadata={'source': 'gutenberg_ebooks_modified.csv', 'row': 0}, page_content=': 0\nindex: 0\nEbook ID: 8600\nAuthor: Zola, Émile, 1840-1902\nTitle: L\'Assommoir\nCredits: John Bickers, Dagny and David Widger\nSummary: "L\'Assommoir" by Émile Zola is a novel written during the late 19th century, an era characterized by the realism movement in literature. The book explores the struggles of Gervaise, a laundress trying to build a life for herself and her children amidst the oppressive and often brutal conditions of working-class Paris. The story highlights themes of poverty, domestic strife, and the impact of alcoholism on individuals and families.  The opening of "L\'Assommoir" presents Gervaise in a state of despair, anxiously awaiting the return of Lantier, her partner, who has been increasingly unfaithful and irresponsible. The narrative depicts her emotional turmoil as she reflects on their meager living conditions and the struggles of single motherhood. Gervaise\'s observa

In [54]:
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.

Question: {question}

Context: {context}

Answer:
"""

In [61]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_template(prompt_template)

# Pour faire une string contenant tous les documents retournés par le retriever, séparés par 2 retours à la ligne
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(), # la question sera parsée comme le premier argument lors de l'invocation
    }
    | prompt
    | llm
    | StrOutputParser()
)

response = qa_chain.invoke('What are the main characters of "L\'Assommoir"?')

In [60]:
def vector_search_tool(args):
    result = vectorstore.similarity_search({"query": args['question']})
    return result
result
    chain_result = vector_graph_chain.invoke({
        "query": args['question']},
    )
    chain_result = "do some thing to chain_re"
    return chain_result

'"L\'Assommoir" by Émile Zola is a novel written during the late 19th century, an era characterized by the realism movement in literature. The book explores the struggles of Gervaise, a laundress trying to build a life for herself and her children amidst the oppressive and often brutal conditions of working-class Paris. The story highlights themes of poverty, domestic strife, and the impact of alcoholism on individuals and families.  The opening of "L\'Assommoir" presents Gervaise in a state of despair, anxiously awaiting the return of Lantier, her partner, who has been increasingly unfaithful and irresponsible. The narrative depicts her emotional turmoil as she reflects on their meager living conditions and the struggles of single motherhood. Gervaise\'s observations of the bustling street below reveal the harsh realities faced by workers in the city, and through her interactions with Lantier, the text illustrates the complexities of love, frustration, and survival in a difficult soci

In [62]:
response

'The main characters of "L\'Assommoir" are Gervaise, a laundress, and Lantier, her partner.'